# iALM Stage 1: Asset Return Simulation

This notebook visualizes the stochastic drivers for the iALM model.
We simulate two core processes described in the paper:
1. **Geometric Brownian Motion (GBM)** for asset classes like Equities.
2. **Geometric Ornstein-Uhlenbeck (GOU)** for mean-reverting rates like Inflation and Interest Rates.

In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt

# Add parent directory to path to import src
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

from ialm.engines import iALM_Simulator

# Configuration
np.random.seed(42)
YEARS = 20
DT = 1/12
STEPS = int(YEARS / DT)
SCENARIOS = 50

## 1. Simulating Equity Markets (GBM)
Using parameters typical for a diversified equity index (Drift=7%, Vol=15%).

In [ ]:
sim = iALM_Simulator(dt=DT)

equity_paths = sim.simulate_gbm(
    S0=100, 
    mu=0.07, 
    sigma=0.15, 
    n_steps=STEPS, 
    n_scenarios=SCENARIOS
)

plt.figure(figsize=(10, 6))
plt.plot(equity_paths, alpha=0.2, color='blue')
plt.plot(np.mean(equity_paths, axis=1), color='black', linewidth=2, label='Mean Path')
plt.title(f"Domestic Equity Scenarios (GBM) - {SCENARIOS} Scenarios")
plt.xlabel("Months")
plt.ylabel("Index Value")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 2. Simulating Inflation (GOU)
Inflation is modeled as mean-reverting. We use a GOU process reverting to 3% (1.03).
Note the "pull" back to the mean when scenarios drift too far.

In [ ]:
inflation_paths = sim.simulate_gou(
    R0=1.03,      # Initial Inflation 3%
    alpha=0.03,   # Long term mean parameter
    beta=0.5,     # Speed of mean reversion
    sigma=0.01,   # Volatility of inflation
    n_steps=STEPS, 
    n_scenarios=SCENARIOS
)

# Convert to percentage for plotting (e.g. 1.03 -> 3.0%)
inflation_pct = (inflation_paths - 1.0) * 100

plt.figure(figsize=(10, 6))
plt.plot(inflation_pct, alpha=0.2, color='red')
plt.axhline(3.0, color='black', linestyle='--', label='Long Term Mean (3%)')
plt.title("Stochastic Inflation Scenarios (GOU)")
plt.xlabel("Months")
plt.ylabel("Inflation Rate (%)")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()